## Logistic Regression and Gradient Descent

### The task: Binary classification

#### Generate data points in 2D to work with:

In [1]:
import toy_data
import bokeh.io
bokeh.io.output_notebook()

gm = toy_data.GaussianMixture(n_class=2)
toy_data.visualize_2D(gm.Classes, gm.class_colors)

Loading BokehJS ...

### Logistic regression:

#### The model:
$$
Y|X \sim \text{Bernolli}( \frac{1}{1+e^{-wX}})
$$
We assume the class labels are 0 and 1, and is [Bernolli distributed](https://en.wikipedia.org/wiki/Bernoulli_distribution).
$\def\logi{\frac{1}{1+e^{-wX}}}$
We also assume given $X$, the probability of getting a 1 is the _logistic function_ $\logi$. I will write a detailed introduction to the rationale of it later, but intuitively we can transfrom $y = \logi$ to $\log{y} + \log{1-y} = wX$, where we map the probability of getting a both answers to a linear transfromation of $X$ by $w$ using $\log$ function.

#### Maximun Likelihood Estimation(MLE):

Given the sampled data, we can estimate linear parameter $w$ using maximumize likelihood estimation, where we find the most possible $w$ that generates the data.


* First we find log-likelihood, which is equivalent to likelihood if we want to maximize it: 

$$ 
\begin{align*}
    &\log{P(Y_1, Y_2, \ldots, Y_n\mid X_1, X_2, \ldots, X_n, w)}    \\
    =& \log{\prod_{i=1}^{n} {P(Y_i \mid X_i, w)}} \\
    =& \sum_{i=1}^{n}{\log{P(Y_i \mid X_i, w)}} \\
    & \text{(Assume \( Y_i\) is bernolli distrubuted with \( p=\frac{1}{1+e^{-wX}}\) )} \\
    =& \sum_{i=1}^{n}{\log{P(Y_i=1 \mid X_i, w)^{I_{Y_i=1}}P(Y_i=0 \mid X_i, w)^{I_{Y_i=0}}}} \\
    =& \sum_{i=1}^{n}{\log{p^{I_{Y_i=1}}(1-p)^{I_{Y_i=0}}}} \\
    =& \sum_{i=1}^{n}{{I_{Y_i=1}}\log{p} +{I_{Y_i=0}}\log{(1-p)}} \\
    =& \sum_{i=1}^{n}{Y_i\log{p} + (1-Y_i)\log{(1-p)}} \\
    =& \sum_{i=1}^{n}{Y_i\log{\frac{p}{1-p}} + \log{(1-p)}} \\
    =& \sum_{i=1}^{n}{Y_iwX_i + \log{\frac{e^{-wX_i}}{1+e^{-wX_i}}}} \\
    =& \sum_{i=1}^{n}{Y_iwX_i - wX_i - \log{(1+e^{-wX_i})}} \\
\end{align*}
$$

Hence we are finding
$$
\max_{w}{\sum_{i=1}^{n}{Y_iwX_i - wX_i - \log{(1+e^{-wX_i})}}}
$$

### Gradient Descent

From MLE, we have the following optimization problem:
$$
   \max_{w}{\sum_{i=1}^{n}{Y_iwX_i - wX_i - \log{(1+e^{-wX_i})}}} 
$$

We can use gradient ascent to iteratively find the maximum, where we calculate the gradient of the likelihood in each step, and update $w$ by $ w_{t+1} = w_{t} + \nabla_w$ 

* Calculating the Gradient: 

$$
\begin{align*}
    & \nabla_w{(\sum_{i=1}^{n}{Y_iwX_i - wX_i - \log{(1+e^{-wX_i})}})} \\
    =& \sum_{i=1}^{n} Y_iX_i - X_i + \frac{X_ie^{-wX_i}}{1+e^{-wX_i}}\\
%    =& \sum_{i=1}^{n} Y_iX_i - X_i\frac{1}{1+e^{-wX_i}}\\
    & \text{(Let}\hat{Y}_i = \frac{1}{1+e^{-wX_i}}) \\
    =& \sum_{i=1}^{n} (Y_i-\hat{Y}_i) X_i\\
\end{align*}
$$

* Hence we can find the maximum by: 

$$
    w_{t+1} = w_{t} + \alpha \sum_{i=1}^{n} (Y_i-\hat{Y}_i) X_i
$$

, where $\alpha$ is the learning rate

### Numpy implementation:

We first define the logistic function and two ways to calculate the likelihood as Python functions

In [2]:
import numpy as np
import numpy.random as rnd

def logistic(_x, _w):  
    # _x : numpy 2DArray
    # _w : numpy 1DArray
    # _w.size = _x.shape[1]
    return 1/(1 + np.exp(-_x.dot(_w)))

def log_likelihood(_X, _Y, _w):
    wX = _X.dot(_w)
    return (_Y - 1).dot(wX) - np.sum(np.logaddexp(0, -wX))
    
def log_likelihood2(_Y, _Yhat):
    _p = (1 - _Y)*(1 - _Yhat) + _Y*_Yhat
    return np.sum(np.log(_p))

and we add a column of 1 to $X$ to simulate $wX + b$ by $w[X:1]$

In [3]:
add_offset = lambda X: np.hstack((X, np.ones((X.shape[0], 1))))
X_with_offset = add_offset(gm.tr.X)

* Then we implement gradient ascent on ```w```

Note that we dynamically adjust the learning rate ```lr``` using [backtracking](https://en.wikipedia.org/wiki/Backtracking_line_search) with parameter ```beta``` and ```alpha```.

The for loop will end if maximun iteration is reached or the error converges.

In [4]:
max_iter = 1000
min_err_change = 0.0001

w = rnd.uniform(-1, 1, X_with_offset.shape[1])

lr = 0.1  # Learning rate
beta = 0.8 # Backtracking
alpha = 0.5 # Backtracking


err0 = None

for _ in range(max_iter):
    pred_lbs = logistic(X_with_offset, w)
    diff_lb = gm.tr.y[:,0] - pred_lbs
    err = np.sum(np.abs(diff_lb))
    
    _lll = log_likelihood(X_with_offset, gm.tr.y[:,0], w)
    # print((lr, err, _lll, _lll2, w))
    if err0:
        if abs(err - err0) < min_err_change:
            print(_)
            break
    err0 = err
    grad = diff_lb.dot(X_with_offset)
    lr = 0.1  # Learning rate
    while log_likelihood(X_with_offset, gm.tr.y[:,0], w+lr*grad) < _lll + lr*alpha*grad.dot(grad):
        lr *= beta
    w += lr*grad

150


#### Visualization of the classification results

In [5]:
def regress(_x, _w):
    _y = logistic(add_offset(_x), _w)
    _Y = np.column_stack((_y, 1 - _y))
    return _Y

toy_data.visualize_2D(gm.Classes, gm.class_colors, classifyF=lambda x: regress(x, w), res=80)

In [5]:

import tensorflow as tf

x_dim = 2


io_tf = lambda dim: tf.placeholder(tf.float32, dim)
var_tf = lambda dim: tf.Variable(dim)
def tf_logistic(_X):  
    return 1/(1 + tf.exp(-_X))
with tf.device('/gpu:0'):
    x = io_tf([None, x_dim])
    y = io_tf([None, 1])
    W = var_tf(tf.zeros([x_dim, 1]))
    b = var_tf(tf.zeros([1]))
    y_ = tf_logistic(tf.matmul(x, W) + b)
    loglikelihood = tf.reduce_sum(tf.log((1-y_)*(1-y) + y*y_))
    dydW = tf.gradients(loglikelihood, W)
    dydb = tf.gradients(loglikelihood, b)

lr = 0.1  # Learning rate
beta = 0.8 # Backtracking
alpha = 0.5 # Backtracking
data_X = gm.tr.X
data_y = np.array([gm.tr.y[:, 0]]).T
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    writer = tf.train.SummaryWriter('summary', sess.graph)
    sess.run(tf.initialize_all_variables())
    for i in range(50):
        l = sess.run(loglikelihood, {x:data_X, y:data_y})
        db = sess.run(dydb, {x:data_X, y:data_y})
        dw = sess.run(dydW, {x:data_X, y:data_y})

        W_old = W.eval()
        b_old = b.eval()
        while True:
            W.assign(W_old + lr*dw[0]).eval()
            b.assign(b_old + lr*db[0]).eval()
            ln = sess.run(loglikelihood, {x:data_X, y:data_y})
            norm_grad = np.sum(dw[0]**2) + db[0]**2
            if ln < l + lr*alpha*norm_grad:
                lr *= beta
            else:
                break
        
    def classifier(_X):
        _y = sess.run(y_, feed_dict={x: _X})
        _Y = np.column_stack((_y, 1 - _y))
        return _Y

    toy_data.visualize_2D(gm.Classes, gm.class_colors, classifyF=classifier, res=100)